# 2.3 텍스트 유사도

다음과 같은 질문을 한다고 생각해보자

- 이 노래 누가 만들었어?
- 지금 나오는 노래의 작곡가가 누구야?

위 두 문장은 똑같은 의미이지만 인공지능 스피커에는 단순하게 다른 문장으로 인식될 것이다.
따라서 각기 다른 대답을 만들어야 하지만 효율성을 위해 비슷한 의미를 가진 문장에 대해서는 같은 대답을 준비 할 수 있을 것이다.
이때 문장이 유사한지 측정해야하며, 이때 사용하는 것이 텍스트 유사도 측정 방법이다.

텍스트 유사도란 말그대로 텍스트가 얼마나 유사한지를 표현하는 방식 중 하나다.
앞에서 예로든 두 문장은 다른 구조의 문장이지만 의미는 비슷하기 때문에 유사도가 높다고 판단할 수 있다.
유사도를 판단하는 척도는 매우 주관적이기 때문에 데이터를 구성하기가 쉽지 않고 정량화 하는 데 한계가 있기 때문에 이를 최대한 정량화해서 모델을 만드는 것이 중요하다.

# 문장 백터화 진행

유사도를 측정하기 위하서는 먼저 단어를 벡터화 하여야한다.
여기서는 TF-IDF를 통해 벡터화한다.

In [2]:
import numpy as np

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 2개의 임의의 텍스트 생성
sent = ("휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.", "폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니 다.") 

# 객체 생성
tfidf_vectorizer = TfidfVectorizer()
# 문장 벡터화 진행
tfidf_matrix = tfidf_vectorizer.fit_transform(sent) 

idf = tfidf_vectorizer.idf_
# 각 수치에 대한 값 시각화
print(dict(zip(tfidf_vectorizer.get_feature_names(), idf)))

# 값이 작을수록 자주 등장한다는 것을 알 수 있다.

{'갑작스런': 1.4054651081081644, '내일': 1.4054651081081644, '놀러왔다가': 1.4054651081081644, '망연자실': 1.4054651081081644, '반가운': 1.4054651081081644, '서쪽': 1.4054651081081644, '소식': 1.4054651081081644, '오늘': 1.4054651081081644, '으로': 1.4054651081081644, '이어졌는데요': 1.4054651081081644, '인해': 1.4054651081081644, '있습니': 1.4054651081081644, '있습니다': 1.4054651081081644, '중심': 1.4054651081081644, '폭염': 1.0, '피해서': 1.4054651081081644, '하고': 1.4054651081081644, '휴일': 1.0}


# 유사도의 예

 1. 자카드 유사도
 두 문장을 각각 단어의 집합으로 만든 뒤 집합을 통해 유사도를 측정하는 방식
 두 집합의 교집합인 공통된 단어의 개수를 두집합의 합집합, 즉 전체 단어의 수로 나누면 된다.
 
 2. 코사인 유사도
 두개의 벡터값에서 코사인 각도를 구하는 방법이다.
 다른 유사도 접근법에 비해 일반적으로 성능이 좋다.
 두 벡터간의 각도를 구하는 것이기 때문에 방향성의 개념이 더해지기 떄문이다.(두 문장이 유사하다면 같은 방향을 가리킨다.)
 
 3. 유클리디안 유사도
 가장 기본적인 거리를 측정하는 유사도 공식
 두 점 사이의 최단 거리를 구하는 접근법이다.
 단, 거리를 구하는 것이기 때문에 1보다 큰 값이 나오므로 값을 제한해야한다.
 보통 0과 1사이의 값으로 제한하며 방법은 벡터화한 문장을 일반화한 후 다시 유클리디언 유사도를 측정하면 된다.
 
 4. 멘하탄 유사도
 사각형 격자로 이뤄진 지도에서 출발점에서 도착점까지를 가로지르지 않고 갈 수 있는 최단거리를 구하는 공식이다.
 이또한 유클리디안과 같이 L1정규화 방법을 사용하여야 한다.

In [7]:
# 1. 자카드 유사도
from sklearn.metrics import jaccard_similarity_score

# jaccard_similarity_score(tfidf_matrix[0:1], tfidf_matrix[1:2])
jaccard_similarity_score(np.array([1,1,0,0]), np.array([1,1,0,2]))

ImportError: cannot import name 'jaccard_similarity_score' from 'sklearn.metrics' (C:\Users\jch\anaconda3\envs\pr_tensorflow\lib\site-packages\sklearn\metrics\__init__.py)

In [8]:
# 2. 코사인 유사도

from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 구해보자
cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

array([[0.11304078]])

In [9]:
# 3. 유클리디안 유사도

from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])

# 정규화

import numpy as np

def l1_normalize(v):
    norm = np.sum(v)
    return v / norm

tfidf_norm_l1 = l1_normalize(tfidf_matrix)
euclidean_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.21243357]])

In [10]:
# 4. 맨하탄 유사도

from sklearn.metrics.pairwise import manhattan_distances

manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.85654185]])

이번 실습에서는 맨하탄 유사도로 측정했을 때 유사도가 가장 높게 나왔다.
측정 방법에 따라 크게 유사도가 달라질 수 있으므로 의도하고자 하는 방향에 맞는 유사도 측정 방법을 고르는 것이 매우 중요하다.